In [8]:
# Import libraries here
import inspect
import time
import warnings

import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import seaborn as sns
from pymongo import MongoClient
from sklearn.metrics import mean_absolute_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA

In [9]:
client = MongoClient(host='localhost', port=27017)
db = client['air-quality']
dar = db['dar-es-salaam']

In [11]:
sites = dar.distinct('metadata.site')
sites

In [ ]:
result = dar.count_documents({'metadata.site':11})
readings_per_site = list(dar.aggregate(
    [{'$group': {'_id': '$metadata.site', 'count':{'$count':{}}}}]
))
readings_per_site

In [13]:
def wrangle(collection):

    results = collection.find(
        {"metadata.site": 11, "metadata.measurement": "P2"},
        projection={"P2": 1, "timestamp": 1, "_id": 0},
    )

    # Read results into DataFrame
    df = pd.DataFrame(list(results)).set_index("timestamp")

    # Localize timezone
    df.index = df.index.tz_localize("UTC").tz_convert("Africa/Dar_es_Salaam")

    # Remove outliers
    df = df[df["P2"] < 100]

    # Resample and forward-fill
    y = df['P2'].resample('1H').mean().fillna(method='ffill')

    return y

In [ ]:
y = wrangle(dar)
y.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
fig = px.line(y, x=y.index, y='P2')
fig.show()
# Don't delete the code below 👇
plt.savefig("images/3-5-5.png", dpi=150)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
fig = px.line(y.rolling(168).mean(), x=y.rolling(168).mean().index, y='P2')
fig.show()
# Don't delete the code below 👇
plt.savefig("images/3-5-6.png", dpi=150)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
plot_acf(y, ax=ax)
plt.xlabel('Lag [hours]')
plt.ylabel('Correlation Coefficient')
# Don't delete the code below 👇
plt.savefig("images/3-5-7.png", dpi=150)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
plot_pacf(y, ax=ax)
plt.xlabel('Lag [hours]')
plt.ylabel('Correlation Coefficient')
# Don't delete the code below 👇
plt.savefig("images/3-5-8.png", dpi=150)


In [ ]:
cutoff_test = int(len(y) * 0.9)
y_train = y.iloc[:cutoff_test]
y_test = y.iloc[cutoff_test:]
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

In [ ]:
y_train_mean = y_train.mean()
y_pred_baseline = [y_train_mean] * len(y_train)
mae_baseline = mean_absolute_error(y_train, y_pred_baseline)

print("Mean P2 Reading:", y_train_mean)
print("Baseline MAE:", mae_baseline)

In [ ]:
p_params = range(1, 31)
maes = []
for p in p_params:
    start_time = time.time()
    model = AutoReg(y_train, lags=p, old_names=False).fit()
    elapsed_time = round(time.time() - start_time, 2)
    print(f'Trained AR Model {p} in {elapsed_time} seconds.')
    y_pred = model.predict().dropna()
    mae = mean_absolute_error(y_train.loc[y_pred.index], y_pred)
    maes.append(mae)
    pass
mae_series = pd.Series(maes, name="mae", index=p_params)
mae_series

In [ ]:
best_p = 27
best_model = AutoReg(y_train, lags=best_p, old_names=False).fit()

In [ ]:
y_train_resid = (y_train - y_pred).dropna()
y_train_resid.name = "residuals"
y_train_resid.head()

In [ ]:
# Plot histogram of residuals
plt.hist(y_train_resid)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Best Model, Training Residuals')
# Don't delete the code below 👇
plt.savefig("images/3-5-14.png", dpi=150)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
plot_acf(y_train_resid, ax=ax)
# Don't delete the code below 👇
plt.savefig("images/3-5-15.png", dpi=150)


In [ ]:
y_pred_wfv = pd.Series()
history = y_train.copy()
for i in range(len(y_test)):
    model = AutoReg(history, lags=best_p, old_names=False).fit()
    next_pred = model.forecast()
    y_pred_wfv = y_pred_wfv.append(next_pred)
    history = history.append(y_test[next_pred.index])
y_pred_wfv.name = "prediction"
y_pred_wfv.index.name = "timestamp"
y_pred_wfv.head()

In [ ]:
df_pred_test = pd.DataFrame(
    {'y_test':y_test, 'y_pred_wfv':y_pred_wfv}, index=y_test.index
)
fig = px.line(df_pred_test)
fig.update_layout(
    title="Dar es Salaam, WFV Predictions",
    xaxis_title="Date",
    yaxis_title="PM2.5 Level",
)
# Don't delete the code below 👇
fig.write_image("images/3-5-18.png", scale=1, height=500, width=700)

fig.show()